In [2]:
from typing import List, Tuple, Dict
import heapq
import numpy as np

def create_node(position: Tuple[int, int], g: float = float('inf'), h: float = 0.0, parent: Dict = None) -> Dict:
    return {
        'position': position,
        'g': g,
        'h': h,
        'f': g + h,
        'parent': parent
    }

def calculate_heuristic(a: Tuple[int, int], b: Tuple[int, int]) -> float:
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

def get_valid_neighbors(grid: np.ndarray, position: Tuple[int, int]) -> List[Tuple[int, int]]:
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_x, new_y = position[0] + dx, position[1] + dy
        if 0 <= new_x < grid.shape[0] and 0 <= new_y < grid.shape[1] and grid[new_x, new_y] == 0:
            neighbors.append((new_x, new_y))
    return neighbors

def reconstruct_path(node: Dict) -> List[Tuple[int, int]]:
    path = []
    while node:
        path.append(node['position'])
        node = node['parent']
    return path[::-1]

def find_path(grid: np.ndarray, start: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
    start_node = create_node(position=start, g=0, h=calculate_heuristic(start, goal))
    
    open_list = [(start_node['f'], start)]
    open_dict = {start: start_node}
    closed_set = set()

    while open_list:
        _, current_pos = heapq.heappop(open_list)
        current_node = open_dict[current_pos]

        if current_pos == goal:
            return reconstruct_path(current_node)

        closed_set.add(current_pos)

        for neighbor_pos in get_valid_neighbors(grid, current_pos):
            if neighbor_pos in closed_set:
                continue

            tentative_g = current_node['g'] + calculate_heuristic(current_pos, neighbor_pos)

            if neighbor_pos not in open_dict:
                neighbor = create_node(
                    position=neighbor_pos,
                    g=tentative_g,
                    h=calculate_heuristic(neighbor_pos, goal),
                    parent=current_node
                )
                heapq.heappush(open_list, (neighbor['f'], neighbor_pos))
                open_dict[neighbor_pos] = neighbor
            elif tentative_g < open_dict[neighbor_pos]['g']:
                neighbor = open_dict[neighbor_pos]
                neighbor['g'] = tentative_g
                neighbor['f'] = tentative_g + neighbor['h']
                neighbor['parent'] = current_node

    return []  # No path found


In [3]:
from typing import List, Tuple, Dict
import heapq

def create_node(position: Tuple[int, int], g: float = float('inf'), h: float = 0.0, parent: Dict = None) -> Dict:
    return {
        'position': position,
        'g': g,
        'h': h,
        'f': g + h,
        'parent': parent
    }

def calculate_heuristic(a: Tuple[int, int], b: Tuple[int, int]) -> float:
    return ((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) ** 0.5

def get_valid_neighbors(grid: List[List[int]], position: Tuple[int, int]) -> List[Tuple[int, int]]:
    neighbors = []
    for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_x, new_y = position[0] + dx, position[1] + dy
        if 0 <= new_x < len(grid) and 0 <= new_y < len(grid[0]) and grid[new_x][new_y] == 0:
            neighbors.append((new_x, new_y))
    return neighbors

def reconstruct_path(node: Dict) -> List[Tuple[int, int]]:
    path = []
    while node:
        path.append(node['position'])
        node = node['parent']
    return path[::-1]

def a_star(grid: List[List[int]], start: Tuple[int, int], goal: Tuple[int, int]) -> Tuple[float, List[Tuple[int, int]]]:
    start_node = create_node(position=start, g=0, h=calculate_heuristic(start, goal))
    
    open_list = [(start_node['f'], start)]
    open_dict = {start: start_node}
    closed_set = set()

    while open_list:
        _, current_pos = heapq.heappop(open_list)
        current_node = open_dict[current_pos]

        if current_pos == goal:
            path = reconstruct_path(current_node)
            return current_node['g'], path

        closed_set.add(current_pos)

        for neighbor_pos in get_valid_neighbors(grid, current_pos):
            if neighbor_pos in closed_set:
                continue

            tentative_g = current_node['g'] + calculate_heuristic(current_pos, neighbor_pos)

            if neighbor_pos not in open_dict:
                neighbor = create_node(
                    position=neighbor_pos,
                    g=tentative_g,
                    h=calculate_heuristic(neighbor_pos, goal),
                    parent=current_node
                )
                heapq.heappush(open_list, (neighbor['f'], neighbor_pos))
                open_dict[neighbor_pos] = neighbor
            elif tentative_g < open_dict[neighbor_pos]['g']:
                neighbor = open_dict[neighbor_pos]
                neighbor['g'] = tentative_g
                neighbor['f'] = tentative_g + neighbor['h']
                neighbor['parent'] = current_node

    return float('inf'), []  # No path found

# Example usage
if __name__ == "__main__":
    # Define the grid (0 for open cells, 1 for obstacles)
    grid = [
        [0, 0, 0, 1, 0],
        [0, 1, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0]
    ]

    start = (0, 0)
    goal = (4, 4)

    cost, path = a_star(grid, start, goal)

    if path:
        print(f"Path found with cost {cost:.2f}:")
        print(" -> ".join(str(pos) for pos in path))
    else:
        print("No path found")

    # Visualize the path on the grid
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if (i, j) == start:
                print("S", end=" ")
            elif (i, j) == goal:
                print("G", end=" ")
            elif (i, j) in path:
                print("*", end=" ")
            elif cell == 1:
                print("#", end=" ")
            else:
                print(".", end=" ")
        print()


Path found with cost 8.00:
(0, 0) -> (0, 1) -> (0, 2) -> (1, 2) -> (2, 2) -> (3, 2) -> (4, 2) -> (4, 3) -> (4, 4)
S * * # . 
. # * # . 
. . * # . 
# # * # . 
. . * * G 


In [ ]:
a =20
b = 30

a = a+b - (b=a)

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3403351489.py, line 4)